## <center>股票之預測</center>

- @author: Vincent Yau
- @version: 1.0 | 2018/05/21
- @email: wenhengqiu@gmail.com

### 数据前处理
- 从一堆数据中获得与台股2912相关的文章，分为2016年及2017年
- 需要解决数据中NaN的问题，尤其是用正则匹配中文字后，数据为空的情况
- 我选择的处理方法：
    - 1、对DataFrame中的NaN，采用dropna()方法
    - 2、对于分词中的空值，赋予一个生字string（不是一个好的方法，但因为我们选好了特征词，所以一个字符对结果影响不大[说明还是有影响]）
    - 2、对于terms的tfidf的缺失，采用赋予0.0001（不是一个好方法）

### 程式逻辑整理
- 讀取某一年的所有文章，獲得所有的日期，存為 dateList
- 獲得某一天的所有文章，每篇文章都作為list的元素保存起來 articlesList，然後直接存入{日期:articlesLsit}字典中
    - 這裡要注意的是，盡量對數據只進行一次讀取，一次讀取多次使用，避免因為頻繁讀取造成了資源浪費
- 對articlesList中的每一篇文章進行分詞處理，單獨計算其 Term Frequency，保存成 Dictionary
    - 使用jieba分詞，而不用N-gram是為了提高分詞的準確性，且可針對經濟新聞定制Stop Words列表
    - 採用jieba之前，對傳入的characters進行正則匹配，獲取其中所有的中文
    - 中文正則表達式為[^\u4e00-\u9fa5]，故當string無中文時會產生NaN，解決方法參考數據前處理
- 計算某一天所有文章詞的tf-idf，整合成一個dict保存起來
- 獲得特征詞表（1000個詞）
- 將每天的文章按照 [特征詞 + tfidf]進行表示，將所有的天數的數據都append到一個List中

### 特征詞的獲得
- 將所有的文章，根據股票的漲跌判斷，分為「看漲文章」「看跌文章」
- 分別將兩類文章進行Features Selection（特征詞抽取），這裡抽取的依據是TF-IDF，目測抽取1000個詞
- 注意特征詞必須要有順序

### 股票漲跌判斷
- 若第x天的股價，與x+n天的股價存在一定比例的漲or跌，則將x天的文章列為漲or跌文章（估計我們會分為漲、跌、不漲不跌）
- Actually，這裡直接將漲跌的label與日期對應起來即可
- 漲跌的判斷在Machine learning中，屬於對數據進行標標籤，因此，我們的模型屬於：監督式學習
- 理論上，若將每篇文章都進行此標記，則會增Traning Dataset，但因數據文章有許多comments，如此操作會造成大量的稀疏數據
- 將某天的所有文章，用list合併一起，并最終用「特征詞：tfidf」進行表示，是現今採用的方式

### 分類器的構建
- MultinomialNB
- Sppport Vector Machine
- GaussianNB

In [1]:
import re
import jieba
import pandas as pd
import numpy as np
import jieba.analyse
from sklearn import svm
from pandas import DataFrame, Series
from sklearn.datasets.base import Bunch
from sklearn.naive_bayes import MultinomialNB, GaussianNB

## 1、分詞與計算tfidf

In [2]:
#jieba中文分詞，并獲取tfidf排名前3000詞（未採用），還要解決詞與vocabulary匹配的問題
def jieba_Chinese(articles):
    all_articles2 = []
    #使用正则选取文章中汉字的部分, 使用str()是为了防止TypeError: expected string or bytes-like object
    new_article = re.sub(r'[^\u4e00-\u9fa5]', '', str(articles))
    if new_article == '':
        new_article = '㐓'
#     new_article2 = jieba.analyse.extract_tags(new_article, topK=3000, withWeight=True, allowPOS=())
    new_article2 = jieba.cut(new_article, cut_all=False)
    for word in new_article2:
        all_articles2.append(word)
    articles_dataFrame = DataFrame(all_articles2)[0].value_counts().to_dict()
    return articles_dataFrame

#傳入某天的aticlesList，獲得分詞后的terms和其term frequency
def get_termList_forArticles(aticlesList):
    termList = []
    for sentens in aticlesList:
        oneArticleTerm = jieba_Chinese(sentens)
        termList.append(oneArticleTerm)
    return termList

#Calculate tf(means Collection Frequency)、Document Frequency
def getFrequency(termList):
    #Store all terms' Document Frequency into allTermDocFre
    allTermDocFre = {}
    
    #Store Collection Frequency for terms into termFrequency
    termFrequency = {}

    for content in termList:
        for term in content:
            if term in allTermDocFre:
                termFrequency[term] = termFrequency[term] + content[term]
                allTermDocFre[term] += 1
            else:
                termFrequency[term] = content[term]
                allTermDocFre[term] = 1
    return allTermDocFre, termFrequency

#Convert list to Series
def getSeries(list, names, indexName):
    h = pd.Series(list, name=names)
    h.index.name = indexName
    return h.reset_index()

#tf-idf
def calTfidf(allTermDocFre, termFrequency, records):
    # The bank's counts is 1830 less than the number of all articles beacause we delete some terms(tf<3 in single article)
    h1 = getSeries(allTermDocFre, 'df', 'Term')
    h2 = getSeries(termFrequency, 'tf', 'Term')
    #这里可以控制输出terms的长度
    data = pd.merge(h1, h2).sort_values(by='tf', ascending=False)
    
    numOfRecords = len(records)
    numOfTerms = len(allTermDocFre)
    #math.log only accepts single float value, So you should use np.log here
    data['tf-idf'] =  np.log(numOfRecords / data['df']) * (data['tf'] / numOfTerms)
    
    #删除某一列的方法为：data.pop('tf')，原数据中不包含'tf'

    #重置索引为Term, 并将以'tf-idf列为基础转为字典
    data = data.set_index(['Term']).to_dict()['tf-idf']    
    return data

#A function of tfidf for the terms from articles
def get_tfidf_for_articlesList(termList):
    testTermsResult = get_termList_forArticles(termList)
    allTermDocFre, termFrequency = getFrequency(testTermsResult)
    terms_tfidf = calTfidf(allTermDocFre, termFrequency, termList)
    return terms_tfidf

## 2、獲得特征詞及構建向量空間

In [3]:
#输入特征词 及 terms及其tfidf构成的字典，输出按照特征词词序排列的tfidf列表
def tfidfList_oneArticle(vocabulary, terms_tfidf):
    list_for_oneArticle = []
    for word in vocabulary:
        if word in terms_tfidf.keys():
            list_for_oneArticle.append(float(terms_tfidf[word]))
        else:
            #如果文章中没有某个词，就将其tfidf赋值为0.0001，随便设置的一个数值（远远小于正常的tf-idf）
            #这里的赋值要记住，最后是用数值做计算，所以不可传入string
            list_for_oneArticle.append(float(0.0001))
    return list_for_oneArticle

#將每天當做字典的key，每天的文章list當做value，存成dict，好處只需要讀一遍所有的文章
def get_article_by_dict(articles, dates):
    #先初始化每個日期，然後添加list
    articles_dict = {}
    for date in dates:
        articles_dict[date] = []

    for index, row in articles.iterrows():
        if row['date'] in dates:
#             print(row['date'])
            articles_dict[row['date']].append(row['article'])
    return articles_dict

#这里每次找出某个date的文章时候，都会扫描一遍aticles_2017，造成效率降低、开销大增，需要解决这个问题
def all_tfidf_list_forModel(dates, articlesDict, vocabualry):
    all_tfidf_list = []
    for date in dates_2017:
        allArticlesforOneDay = articles_dict[date]
        terms_tfidf = get_tfidf_for_articlesList(allArticlesforOneDay)
        list_for_oneArticle = tfidfList_oneArticle(feature_words_selection, terms_tfidf)
        all_tfidf_list.append(list_for_oneArticle)
        print('Caculation of %s is ok' % date)
    return all_tfidf_list

#计算预测的正确率
def trueRate(prediction, label):
    length = len(prediction)
    greatRate = 0
    for i in range(length):
        if prediction[i] == label[i]:
            greatRate += 1
    return greatRate/length
# #传入的值包括每条数据的label，向量词库，每条数据的tfdif
# def data_for_classfier(label, tfidfm, vocabulary):
#     #tfidfm为tfidf的矩阵，即词向量空间实例，到此为止，比较重要的是tfidfm，与vocabulary
#     #测试集与训练集的格式相似，即有相同的词向量空间Vocabulary，但tfidfm不同。
#     bunch = Bunch(label=[], tfidfm=[], vocabulary={})
#     bunch.label = label
#     bunch.tfidfm = tfidfm
#     bunch.vocabulary = vocabulary
#     return bunch

In [44]:
def get_all_tfidf_list(path_aricles, path_vocabulary, path_label): 
    #获得所有文章，去除數據中的NaN值, axis=0代表去掉有空值的行
    articles_2017 = pd.read_excel(path_aricles).dropna(axis=0)

    #获取特征词list
    feature_words_selection = list(pd.read_excel(path_vocabulary, squeeze=True))

    all_label = pd.read_excel(path_label)
    #獲得所有數據的label
    articles_label = list(all_label['label'])
    #獲得所有的日期
    dates_2017 = list(all_label['dates'])

    #獲得{日期:文章list}字典,最后一个月是dates_2017[202:]
    articles_dict = get_article_by_dict(articles_2017, dates_2017)

    #向量化Training Data
    all_tfidf_list = all_tfidf_list_forModel(dates_2017, articles_dict, feature_words_selection)
    return all_tfidf_list, articles_label

----

## 3、構建分類模型 

In [40]:
def stockPrediction(all_tfidf_list, articles_label):    
    #MutilnomiaNB分类方式
    clf = MultinomialNB(alpha=0.001).fit(all_tfidf_list[:200], articles_label[:200])
    #传入的数据是，列为feature的个数，行为某个要分类的文本
    nb_prediction = clf.predict(all_tfidf_list[200:])
    print(nb_prediction)
    nbRate = trueRate(nb_prediction, articles_label[200:])
    print(nbRate)

    #svm分类方式，Support Vector Machine
    svm_clf = svm.SVC().fit(all_tfidf_list[:200], articles_label[:200])
    svm_prediction = svm_clf.predict(all_tfidf_list[200:])
    print(svm_prediction)
    svmRate = trueRate(svm_prediction, articles_label[200:])
    print(svmRate)

    #高斯分类
    gnb_clf = GaussianNB().fit(all_tfidf_list[:200], articles_label[:200])
    gnb_prediction = gnb_clf.predict(all_tfidf_list[200:])
    print(gnb_prediction)
    gnbRate = trueRate(gnb_prediction, articles_label[200:])
    print(gnbRate)

----

In [49]:
path_aricles = '/Users/vincentyau/Documents/台大硕一下资料/大数据与商业分析/final_version/social_d.xlsx'
path_vocabulary = '/Users/vincentyau/Documents/台大硕一下资料/大数据与商业分析/final_version/2912_price_up_term.xlsx'
path_label = '/Users/vincentyau/Documents/台大硕一下资料/大数据与商业分析/final_version/2912_s_17_label.xlsx'

In [ ]:
all_tfidf_list, articles_label = get_all_tfidf_list(path_aricles, path_vocabulary, path_label)

In [52]:
stock_2017 = stockPrediction(all_tfidf_list, articles_label)

['U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U']
0.625
['U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U']
0.625
['D' 'D' 'D' 'D' 'D' 'D' 'U' 'D' 'D' 'D' 'D' 'U' 'D' 'D' 'U' 'U' 'U' 'D'
 'U' 'D' 'D' 'D' 'D' 'D']
0.375


------------------